In [1]:
import pandas as pd
import numpy as np
import random
from random import seed


## Part 1: Import Balance Data 
Simulated balance data is imported from the GitHub repo. 100 balance profiles have been simulated for different types of business and personal accounts.

In [2]:
import pandas as pd
import numpy as np

def read_file(url):

    """
    Takes GitHub url as an argument,
    pulls CSV file located @ github URL.

    """

    url = url + "?raw=true"
    df = pd.read_csv(url, encoding="utf-8")
    return df


# READ FILE FROM GITHUB REPO
url = "https://github.com/john-adeojo/BusinessThroughPersonalPoC/blob/main/GeneratedAccountBalances.csv"
generatedbalance = read_file(url)
generatedbalance.Date = pd.to_datetime(generatedbalance.Date)  # Convert balances to apprpriate date format

In [3]:
generatedbalance.head()

,Unnamed: 0,Date,Account Number,Balance,AccountType_actual,AccountType
0,0,2014-01-02,0,-1.050230e+06,Limited Company,Personal
1,1,2014-01-03,0,-2.042959e+05,Limited Company,Personal
2,2,2014-01-04,0,-2.599986e+05,Limited Company,Personal
3,3,2014-01-05,0,-1.274114e+06,Limited Company,Personal
4,4,2014-01-06,0,-2.269378e+05,Limited Company,Personal


## Part 2: Generate time series features
Create the time series features 

In [4]:
from tsfresh import extract_features


In [5]:
fc_parameters = {
    "variation_coefficient": None,
    "autocorrelation":[{"lag":30}],
    "count_above_mean":None,
    "count_below_mean":None,
    "longest_strike_below_mean":None,
    "longest_strike_above_mean":None,
   # "percentage_of_reoccurring_values_to_all_datapoints":None,
    "skewness":None,
    "kurtosis":None
    
}

# Create two tables, TimeSeriesFeatures: has only the time series features for each account (table index = account number), FeaturesAccounts: has time series features, accountnumbers and Accounttype
TimeSeriesFeatures = extract_features(timeseries_container=generatedbalance, column_id="Account Number", column_sort="Date", column_value="Balance", default_fc_parameters=fc_parameters)
FeaturesAccounts = TimeSeriesFeatures.merge(generatedbalance[['Account Number','AccountType', 'AccountType_actual']], left_index=True, right_on='Account Number', how='inner')

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 25/25 [00:08<00:00,  2.84it/s]


In [6]:
TimeSeriesFeatures.head()

,Balance__variation_coefficient,Balance__autocorrelation__lag_30,Balance__count_above_mean,Balance__count_below_mean,Balance__longest_strike_below_mean,Balance__longest_strike_above_mean,Balance__skewness,Balance__kurtosis
0,-3.216077,-0.093801,171.0,156.0,6.0,9.0,-0.110677,-0.075851
1,-2.485133,-0.038949,157.0,170.0,9.0,6.0,-0.154101,0.215769
2,4.824153,-0.050650,167.0,160.0,8.0,9.0,0.013867,-0.562282
3,-0.919044,-0.038922,165.0,162.0,7.0,11.0,0.057942,0.849858
4,-8.122328,0.085648,163.0,164.0,6.0,8.0,-0.203784,0.651263


In [7]:
FeaturesAccounts.head()

,Balance__variation_coefficient,Balance__autocorrelation__lag_30,Balance__count_above_mean,Balance__count_below_mean,Balance__longest_strike_below_mean,Balance__longest_strike_above_mean,Balance__skewness,Balance__kurtosis,Account Number,AccountType,AccountType_actual
0,-3.216077,-0.093801,171.0,156.0,6.0,9.0,-0.110677,-0.075851,0,Personal,Limited Company
1,-3.216077,-0.093801,171.0,156.0,6.0,9.0,-0.110677,-0.075851,0,Personal,Limited Company
2,-3.216077,-0.093801,171.0,156.0,6.0,9.0,-0.110677,-0.075851,0,Personal,Limited Company
3,-3.216077,-0.093801,171.0,156.0,6.0,9.0,-0.110677,-0.075851,0,Personal,Limited Company
4,-3.216077,-0.093801,171.0,156.0,6.0,9.0,-0.110677,-0.075851,0,Personal,Limited Company


## Part 3: Create K-means clusters
Perform Kmeans clustering on time series features, 
No optimisation has been performed on the clusters 

In [8]:
import sklearn 
from sklearn.cluster import KMeans
# Create clusters for the time series features and write those clusters to a csv.
x = list((TimeSeriesFeatures.columns))
clusters = KMeans(n_clusters = 10, random_state=15).fit_predict(X=FeaturesAccounts[x])
clustersdf = pd.DataFrame(clusters, columns=['cluster'])
clustersdf

,cluster
0,1
1,1
2,1
3,1
4,1
...,...
32695,4
32696,4
32697,4
32698,4


In [9]:
# Create the cluster analysis by merging the clusters to generated balances and removing duplicates.
clusteredTimeSeries = clustersdf.merge(generatedbalance[['AccountType', 'Account Number', 'AccountType_actual']], left_index=True, right_index=True, how='left').drop_duplicates()
clusteredTimeSeries

,cluster,AccountType,Account Number,AccountType_actual
0,1,Personal,0,Limited Company
327,7,Sole Trader,1,Sole Trader
654,6,Limited Company,2,Sole Trader
981,4,Sole Trader,3,Limited Company
1308,9,Personal,4,Sole Trader
...,...,...,...,...
31065,7,Limited Company,95,Limited Company
31392,5,Sole Trader,96,Limited Company
31719,6,Limited Company,97,Personal
32046,1,Sole Trader,98,Sole Trader


## Part 4: Analyse the clusters
Create the cluster analysis

In [10]:
analysis = clusteredTimeSeries.groupby(['cluster', 'AccountType']).size().unstack(fill_value=0)

In [11]:
analysis

AccountType,Limited Company,Personal,Sole Trader
cluster,,,
0,3,7,3
1,0,5,11
2,0,1,0
3,2,0,0
4,1,2,7
5,1,2,13
6,9,0,0
7,2,8,5
8,0,1,0


## Part 5: Decision Criteria 
Create accounts that need to be investigate by applying decision rules based on cluster analysis. The decision rule is to output any Personal account that is a minority class in its cluster. 
Please be aware that decision criteria is hardcoded. We should aim to make this dynamic for production. 

In [12]:
# Ouput decision cirteria
Investigate = clusteredTimeSeries.loc[(clusteredTimeSeries.cluster ==1)|(clusteredTimeSeries.cluster == 9)|(clusteredTimeSeries.cluster == 5)]
Investigate = Investigate.loc[Investigate.AccountType == "Personal"]

In [13]:
Investigate

,cluster,AccountType,Account Number,AccountType_actual
0,1,Personal,0,Limited Company
1308,9,Personal,4,Sole Trader
8175,1,Personal,25,Limited Company
9483,5,Personal,29,Limited Company
15369,1,Personal,47,Limited Company
18639,1,Personal,57,Limited Company
20274,1,Personal,62,Limited Company
22563,5,Personal,69,Limited Company
